In [36]:
import os

import pandas as pd

In [37]:
data_dir = '/Users/rebekahzhang/data/neural_data/processing_check'

progress_file_name = "RZ_progress.csv"  # downloaded from df pipeline by running progress_checker.py
recording_log_name = "recording_log.csv"  # downloaded from recording log from google drive

dj progress check

In [38]:
dj_progress = pd.read_csv(os.path.join(data_dir, progress_file_name))

In [39]:
flow_columns = [
    'Event', 'Synchronization', 'ProbeInsertion', 'ClusteringTask', 
    'PreProcessing', 'SIClustering', 'PostProcessing', 'SIExport', 
    'Clustering', 'CuratedClustering', 'ManualCuration', 'OfficialCuration', 
    'ApplyOfficialCuration'
]
dj_progress['First_X_Column'] = dj_progress[flow_columns].eq('X').idxmax(axis=1)
dj_progress['First_X_Column'] = dj_progress['First_X_Column'].replace('Event', 'Done')

In [40]:
counts = dj_progress['First_X_Column'].value_counts()
print(counts)

First_X_Column
Done              104
SIClustering       11
SIExport           10
ManualCuration      5
Name: count, dtype: int64


adding progress info to recording log

In [41]:
recording_log = pd.read_csv(os.path.join(data_dir, recording_log_name))
recording_log = recording_log.drop(columns=['NIDAQ', 'simultaneous',
       'probe', 'hemisphere', 'depth', 'probe treatment', 'insertion speed',
       'resting time', 'surface', 'extraction speed', 'notes', 'rewards',
       'num trials', 'tw', 'current status', 'final data out'])

In [42]:
recording_log

,date,mouse,insertion_number,region,potential problems,sorting notes
0,2024-07-11,RZ034,0,str,NaN,NaN
1,2024-07-12,RZ034,0,str,D drive ran out of space,NaN
2,2024-07-13,RZ034,0,v1,NaN,assertion error when lauching phy
3,2024-07-13,RZ034,1,str,NaN,should be all good now
4,2024-07-14,RZ034,0,v1,NaN,invalid sorting key
...,...,...,...,...,...,...
166,2025-02-12,RZ070,1,str,pump wasnt properly grounded,NaN
167,2025-02-13,RZ070,0,v1,NaN,"no bug, new phy"
168,2025-02-13,RZ070,1,str,NaN,NaN
169,2025-02-14,RZ070,0,v1,NaN,"no bug, new phy"


In [43]:
dj_progress['date'] = pd.to_datetime(dj_progress['session_datetime']).dt.date
dj_progress['mouse'] = dj_progress['subject']

In [44]:
# Convert 'date' to datetime (handles strings like '2023-10-01' or mixed formats)
recording_log['date'] = pd.to_datetime(recording_log['date'], errors='coerce')
dj_progress['date'] = pd.to_datetime(dj_progress['date'], errors='coerce')

In [45]:
# Convert to integer (if no decimals)
recording_log['insertion_number'] = recording_log['insertion_number'].astype(int)
dj_progress['insertion_number'] = dj_progress['insertion_number'].astype(int)

In [46]:
# Merge with left join on 3 key columns
merged_df = recording_log.merge(
    dj_progress[['mouse', 'date', 'insertion_number', 'First_X_Column']],
    on=['mouse', 'date', 'insertion_number'],
    how='left'
)

# Label unmatched rows
merged_df['First_X_Column'] = merged_df['First_X_Column'].fillna('not_uploaded')

In [47]:
merged_df

,date,mouse,insertion_number,region,potential problems,sorting notes,First_X_Column
0,2024-07-11,RZ034,0,str,NaN,NaN,SIClustering
1,2024-07-12,RZ034,0,str,D drive ran out of space,NaN,not_uploaded
2,2024-07-13,RZ034,0,v1,NaN,assertion error when lauching phy,ManualCuration
3,2024-07-13,RZ034,1,str,NaN,should be all good now,Done
4,2024-07-14,RZ034,0,v1,NaN,invalid sorting key,SIExport
...,...,...,...,...,...,...,...
166,2025-02-12,RZ070,1,str,pump wasnt properly grounded,NaN,Done
167,2025-02-13,RZ070,0,v1,NaN,"no bug, new phy",Done
168,2025-02-13,RZ070,1,str,NaN,NaN,Done
169,2025-02-14,RZ070,0,v1,NaN,"no bug, new phy",Done


In [48]:
merged_df.to_csv(os.path.join(data_dir, 'sessions_cross_checked.csv'))